### Libraries:

In [1]:
!pip install tpot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=56b4dbc73ed244e2813d4f9fa115334a5e4691bc8cb639a2c362039548289f23
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn import preprocessing

import time
from tpot import TPOTRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

### Warnings:

In [3]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message="The objective has been evaluated at this point before.")

### Read dataset:

In [4]:
from google.colab import files
uploaded = files.upload()

Saving data2_filled.csv to data2_filled.csv


In [5]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['data2_filled.csv']))
df

,Study_Folder,Subject_Folder,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,0,214.0,145.0,226.0,226.0,224.0,367.0,229.0,288.0,0.440271,0.124451,0.166400
1,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,1,208.0,137.0,234.0,234.0,226.0,370.0,230.0,288.0,0.351382,0.124451,0.166400
2,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,2,211.0,138.0,230.0,230.0,222.0,370.0,227.0,290.0,0.397633,0.124451,0.166400
3,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,3,212.0,136.0,230.0,230.0,225.0,367.0,227.0,288.0,0.493340,0.124451,0.166400
4,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,4,213.0,141.0,229.0,229.0,223.0,371.0,232.0,288.0,0.450772,0.124451,0.166400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60420,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,139,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763,0.125000,0.166667
60421,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,140,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050,0.125000,0.166667
60422,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,141,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886,0.125000,0.165266
60423,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,142,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049,0.124349,0.165266


In [ ]:
# df = pd.read_csv('data2_filled.csv')
# df

### Drop columns:

In [6]:
df.drop(['Study_Folder','Subject_Folder'], axis = 1, inplace = True)

In [7]:
df

,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,0,0,214.0,145.0,226.0,226.0,224.0,367.0,229.0,288.0,0.440271,0.124451,0.166400
1,0,1,208.0,137.0,234.0,234.0,226.0,370.0,230.0,288.0,0.351382,0.124451,0.166400
2,0,2,211.0,138.0,230.0,230.0,222.0,370.0,227.0,290.0,0.397633,0.124451,0.166400
3,0,3,212.0,136.0,230.0,230.0,225.0,367.0,227.0,288.0,0.493340,0.124451,0.166400
4,0,4,213.0,141.0,229.0,229.0,223.0,371.0,232.0,288.0,0.450772,0.124451,0.166400
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60420,7998,139,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763,0.125000,0.166667
60421,7998,140,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050,0.125000,0.166667
60422,7998,141,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886,0.125000,0.165266
60423,7998,142,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049,0.124349,0.165266


### Define Stimuli, X,Y

In [8]:
df.columns

Index(['Stimuli', 'Frame Number', 'Face_Y', 'Face_X', 'Face_W', 'Face_H',
       'Right_Eye_X', 'Right_Eye_Y', 'Left_Eye_X', 'Left_Eye_Y', 'Face_Angle',
       'X_Predictions', 'Y_Predictions'],
      dtype='object')

In [9]:
features_columns = ['Stimuli', 'Frame Number', 'Face_Y', 'Face_X', 'Face_W','Face_H',
                    'Right_Eye_X', 'Right_Eye_Y', 'Left_Eye_X', 'Left_Eye_Y', 'Face_Angle']

target_columns=['X_Predictions', 'Y_Predictions']

In [10]:
X=df[features_columns]

In [11]:
y=df[target_columns]

In [12]:
X_train_valid= X[X['Stimuli']<= 11]

In [13]:
X_test= X[X['Stimuli']> 11]

In [14]:
y_train_valid= y[X['Stimuli']<= 11]

In [15]:
y_test= y[X['Stimuli']> 11]

### Split data:

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.3,stratify =X_train_valid['Stimuli'] ,random_state=42)

In [17]:
print("Whole dataset :",len(df))
print("X Train size", len(X_train))
print("y Train size", len(y_train))
print("X Valid size", len(X_valid))
print("y Valid size", len(y_valid))
print("X Test size", len(X_test))
print("y Test size", len(y_test))

Whole dataset : 60425
X Train size 23720
y Train size 23720
X Valid size 10167
y Valid size 10167
X Test size 26538
y Test size 26538


In [18]:
print("X Train shape",X_train.shape)
print("y Train shape",y_train.shape)
print("X Valid shape",X_valid.shape)
print("y Valid shape",y_valid.shape)
print("X Test shape",X_test.shape)
print("y Test shape",y_test.shape)

X Train shape (23720, 11)
y Train shape (23720, 2)
X Valid shape (10167, 11)
y Valid shape (10167, 2)
X Test shape (26538, 11)
y Test shape (26538, 2)


In [19]:
X_train['Stimuli'].value_counts().sort_values()/len(X_train), X_valid['Stimuli'].value_counts().sort_values()/len(X_valid)

(0     0.080017
 9     0.083347
 5     0.083474
 2     0.083474
 4     0.083516
 1     0.083600
 10    0.083642
 3     0.083642
 6     0.083685
 7     0.083769
 8     0.083853
 11    0.083980
 Name: Stimuli, dtype: float64,
 0     0.079965
 9     0.083309
 5     0.083505
 2     0.083505
 4     0.083505
 6     0.083604
 10    0.083604
 1     0.083604
 3     0.083702
 7     0.083801
 8     0.083899
 11    0.083997
 Name: Stimuli, dtype: float64)

### Drop columns:

In [20]:
drop_column = ['Stimuli','Frame Number']

In [21]:
X.drop(columns = drop_column, inplace=True)

<ipython-input-21-02a2a226598b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns = drop_column, inplace=True)


In [22]:
X

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
0,214.0,145.0,226.0,226.0,224.0,367.0,229.0,288.0,0.440271
1,208.0,137.0,234.0,234.0,226.0,370.0,230.0,288.0,0.351382
2,211.0,138.0,230.0,230.0,222.0,370.0,227.0,290.0,0.397633
3,212.0,136.0,230.0,230.0,225.0,367.0,227.0,288.0,0.493340
4,213.0,141.0,229.0,229.0,223.0,371.0,232.0,288.0,0.450772
...,...,...,...,...,...,...,...,...,...
60420,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763
60421,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050
60422,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886
60423,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049


In [23]:
X_train.drop(columns = drop_column, inplace=True)

In [24]:
X_train

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
30626,199.0,183.0,205.0,205.0,266.0,340.0,266.0,259.0,0.353743
10603,225.0,187.0,202.0,202.0,260.0,366.0,259.0,286.0,0.469339
28769,110.0,309.0,199.0,199.0,389.0,242.0,391.0,172.0,0.156968
30149,133.0,242.0,206.0,206.0,331.0,271.0,330.0,202.0,0.206556
16125,239.0,160.0,201.0,201.0,238.0,382.0,231.0,310.0,0.550115
...,...,...,...,...,...,...,...,...,...
28521,153.0,195.0,269.0,269.0,302.0,332.0,306.0,237.0,0.230310
17344,148.0,275.0,209.0,209.0,359.0,295.0,345.0,214.0,0.164995
30525,235.0,160.0,225.0,225.0,255.0,393.0,254.0,311.0,0.642546
26050,254.0,163.0,249.0,249.0,262.0,424.0,268.0,337.0,0.501909


In [25]:
X_valid.drop(columns = drop_column, inplace=True)

In [26]:
X_valid

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
127,217.0,146.0,231.0,231.0,230.0,380.0,237.0,296.0,0.381793
32452,228.0,255.0,179.0,179.0,328.0,347.0,328.0,288.0,0.379762
19084,223.0,158.0,210.0,210.0,232.0,367.0,229.0,287.0,0.608490
16347,245.0,155.0,221.0,221.0,249.0,397.0,246.0,320.0,0.688683
16544,195.0,176.0,215.0,215.0,262.0,342.0,264.0,260.0,0.333173
...,...,...,...,...,...,...,...,...,...
22209,197.0,180.0,210.0,210.0,264.0,337.0,264.0,257.0,0.321964
16792,272.0,122.0,261.0,261.0,225.0,454.0,222.0,363.0,1.236665
19581,271.0,120.0,262.0,262.0,226.0,442.0,224.0,358.0,1.236923
9269,174.0,65.0,283.0,283.0,191.0,355.0,187.0,246.0,0.364568


In [27]:
X_test.drop(columns = drop_column, inplace=True)

<ipython-input-27-7ac6106f35b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns = drop_column, inplace=True)


In [28]:
X_test

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
33887,211.0,159.0,236.0,236.0,244.0,369.0,249.0,281.0,0.362482
33888,211.0,157.0,238.0,238.0,243.0,367.0,249.0,283.0,0.379243
33889,210.0,159.0,235.0,235.0,244.0,369.0,249.0,280.0,0.351363
33890,210.0,158.0,236.0,236.0,244.0,368.0,248.0,282.0,0.357945
33891,209.0,158.0,238.0,238.0,244.0,368.0,249.0,282.0,0.360192
...,...,...,...,...,...,...,...,...,...
60420,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763
60421,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050
60422,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886
60423,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049


### MinMaxScaler

In [29]:
minmax_scaler = preprocessing.MinMaxScaler()
X_train_scal = minmax_scaler.fit_transform(X_train)
X_valid_scal = minmax_scaler.transform(X_valid)
X_test_scal = minmax_scaler.transform(X_test)

In [30]:
X_train_scal

array([[0.36533958, 0.40566038, 0.23668639, ..., 0.38970588, 0.39111111,
        0.00370046],
       [0.42622951, 0.41509434, 0.21893491, ..., 0.37254902, 0.45111111,
        0.00538576],
       [0.15690867, 0.70283019, 0.20118343, ..., 0.69607843, 0.19777778,
        0.00083163],
       ...,
       [0.44964871, 0.35141509, 0.35502959, ..., 0.36029412, 0.50666667,
        0.00791098],
       [0.4941452 , 0.35849057, 0.49704142, ..., 0.39460784, 0.56444444,
        0.00586059],
       [0.264637  , 0.01415094, 0.6035503 , ..., 0.02205882, 0.34666667,
        0.00992458]])

In [31]:
X_valid_scal

array([[0.40749415, 0.31839623, 0.39053254, ..., 0.31862745, 0.47333333,
        0.0041094 ],
       [0.43325527, 0.5754717 , 0.08284024, ..., 0.54166667, 0.45555556,
        0.00407979],
       [0.42154567, 0.34669811, 0.26627219, ..., 0.29901961, 0.45333333,
        0.00741446],
       ...,
       [0.53395785, 0.25707547, 0.5739645 , ..., 0.28676471, 0.61111111,
        0.01657654],
       [0.30679157, 0.12735849, 0.69822485, ..., 0.19607843, 0.36222222,
        0.00385827],
       [0.36299766, 0.45518868, 0.10059172, ..., 0.41176471, 0.39777778,
        0.00418931]])

In [32]:
X_test_scal

array([[0.39344262, 0.3490566 , 0.42011834, ..., 0.34803922, 0.44      ,
        0.00382786],
       [0.39344262, 0.34433962, 0.43195266, ..., 0.34803922, 0.44444444,
        0.00407222],
       [0.3911007 , 0.3490566 , 0.41420118, ..., 0.34803922, 0.43777778,
        0.00366576],
       ...,
       [0.62295082, 0.22641509, 0.35502959, ..., 0.2254902 , 0.67333333,
        0.02678145],
       [0.62529274, 0.22641509, 0.34911243, ..., 0.2254902 , 0.66222222,
        0.03251346],
       [0.6206089 , 0.22641509, 0.35502959, ..., 0.22303922, 0.67111111,
        0.03118879]])

### TPOT - Valid:

In [33]:
log_table_v = []

start_time = time.time()

tpot = TPOTRegressor(generations=5, population_size=5, verbosity=2, cv=5, random_state=42)

multioutput_tpot = MultiOutputRegressor(tpot)

multioutput_tpot.fit(X_train_scal, y_train)

y_pred = multioutput_tpot.predict(X_valid_scal)

mse_3 = mean_squared_error(y_valid, y_pred)
mae_3 = mean_absolute_error(y_valid, y_pred)
r2_3 = r2_score(y_valid, y_pred)

end_time = time.time()
elapsed_time = end_time - start_time

minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

log_table_v.append({"Mean Squared Error": mse_3,
                  "Mean Absolute Error": mae_3,
                  "R-squared": r2_3,
                  "Elapsed Time (minutes)": minutes,
                  "Elapsed Time (seconds)": seconds})

print("Mean Squared Error:", mse_3)
print("Mean Absolute Error:", mae_3)
print("R-squared:", r2_3)
print(f"Elapsed time: {minutes} minutes {seconds} seconds")

Optimization Progress:   0%|          | 0/30 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.015836658584928266

Generation 2 - Current best internal CV score: -0.015836658584928266

Generation 3 - Current best internal CV score: -0.015836658584928266

Generation 4 - Current best internal CV score: -0.015836658584928266

Generation 5 - Current best internal CV score: -0.014898614782944516

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=False, max_features=0.7500000000000001, min_samples_leaf=7, min_samples_split=9, n_estimators=100)


Optimization Progress:   0%|          | 0/30 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.008626979440444424

Generation 2 - Current best internal CV score: -0.008265353920956406

Generation 3 - Current best internal CV score: -0.008265353920956406

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=6, min_samples_split=9, n_estimators=100)
Mean Squared Error: 0.010525612022253442
Mean Absolute Error: 0.05816857475388339
R-squared: 0.862387005760419
Elapsed time: 43 minutes 27 seconds


In [34]:
log_table_df = pd.DataFrame(log_table_v)

log_table_df.to_csv('log_table_v_tpot.csv', index=False)

In [39]:
from google.colab import files

files.download('log_table_v_tpot.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### TPOT - Test:

In [35]:
log_table_t = []

start_time = time.time()

tpot = TPOTRegressor(generations=5, population_size=5, verbosity=2, cv=5, random_state=42)

multioutput_tpot = MultiOutputRegressor(tpot)

multioutput_tpot.fit(X_train_scal, y_train)

y_pred = multioutput_tpot.predict(X_test_scal)

mse_3 = mean_squared_error(y_test, y_pred)
mae_3 = mean_absolute_error(y_test, y_pred)
r2_3 = r2_score(y_test, y_pred)

end_time = time.time()
elapsed_time = end_time - start_time

minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

log_table_t.append({"Mean Squared Error": mse_3,
                  "Mean Absolute Error": mae_3,
                  "R-squared": r2_3,
                  "Elapsed Time (minutes)": minutes,
                  "Elapsed Time (seconds)": seconds})

print("Mean Squared Error:", mse_3)
print("Mean Absolute Error:", mae_3)
print("R-squared:", r2_3)
print(f"Elapsed time: {minutes} minutes {seconds} seconds")

Optimization Progress:   0%|          | 0/30 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.015836658584928266

Generation 2 - Current best internal CV score: -0.015836658584928266

Generation 3 - Current best internal CV score: -0.015836658584928266

Generation 4 - Current best internal CV score: -0.015836658584928266

Generation 5 - Current best internal CV score: -0.014898614782944516

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=False, max_features=0.7500000000000001, min_samples_leaf=7, min_samples_split=9, n_estimators=100)


Optimization Progress:   0%|          | 0/30 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.008626979440444424

Generation 2 - Current best internal CV score: -0.008265353920956406

Generation 3 - Current best internal CV score: -0.008265353920956406

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=6, min_samples_split=9, n_estimators=100)
Mean Squared Error: 0.049964429787466555
Mean Absolute Error: 0.17295482829982112
R-squared: 0.3360803173401387
Elapsed time: 43 minutes 13 seconds


In [36]:
log_table_df = pd.DataFrame(log_table_t)

log_table_df.to_csv('log_table_t_tpot.csv', index=False)

In [38]:
from google.colab import files

files.download('log_table_t_tpot.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>